<a href="https://colab.research.google.com/github/nanpolend/365E5/blob/master/%E9%87%91%E8%9E%8D%E9%A2%A8%E6%9A%B4%E7%99%BC%E7%94%9F%E6%A9%9F%E7%8E%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

class GlobalDepressionModel:
    def __init__(self):
        # 模型參數(基於歷史經濟危機數據校準)
        self.factors = [
            'tariff_increase',   # 關稅增幅(百分比點)
            'gdp_growth',       # 全球GDP年增率(%)
            'trade_dependency', # 全球貿易依存度(貿易總額/GDP)
            'debt_gdp_ratio',   # 全球平均政府債務/GDP比率
            'vix',              # 恐慌指數(芝加哥期權交易所波動率)
            'yield_curve'       # 美債殖利率曲線斜率(10年-2年利差)
        ]
        self.model = LogisticRegression()
        self.scaler = StandardScaler()

    def generate_sim_data(self, n_samples=1000):
        """
        生成模擬數據集(因真實危機數據稀缺)
        數據關係基於經濟理論與歷史事件相關性:
        - 關稅上升會降低貿易依存度與GDP增長
        - 殖利率曲線倒掛預示衰退風險
        - 高債務加重財政脆弱性
        """
        np.random.seed(42)
        data = pd.DataFrame({
            'tariff_increase': np.random.uniform(0, 15, n_samples),  # 0-15%關稅增幅
            'gdp_growth': np.random.normal(2.5, 1.5, n_samples),     # 全球GDP增長分布
            'trade_dependency': np.random.uniform(40, 70, n_samples),# 貿易依存度40-70%
            'debt_gdp_ratio': np.random.lognormal(3.5, 0.3, n_samples),
            'vix': np.abs(np.random.normal(15, 8, n_samples)),       # 恐慌指數分布
            'yield_curve': np.random.normal(1.0, 0.8, n_samples)     # 殖利率曲線斜率
        })

        # 危機標籤生成邏輯(非線性組合)
        crisis_risk = (
            0.3 * data['tariff_increase']**1.5 +
            0.4 * (100 - data['trade_dependency']) +
            0.2 * data['debt_gdp_ratio'] +
            0.15 * data['vix'] -
            0.5 * data['yield_curve']**3 -
            2.5 * data['gdp_growth']
        )
        data['depression_label'] = (crisis_risk > np.percentile(crisis_risk, 85)).astype(int)
        return data

    def train_model(self, data):
        """
        訓練危機預測模型
        """
        X = data[self.factors]
        y = data['depression_label']
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)

    def predict_depression_prob(self, scenario):
        """
        預測特定情境下的大蕭條機率
        """
        X_input = pd.DataFrame([scenario])
        X_scaled = self.scaler.transform(X_input)
        return self.model.predict_proba(X_scaled)[0][1]

# 實戰應用 ====================================
# 初始化模型
model = GlobalDepressionModel()

# 生成並訓練模擬數據
sim_data = model.generate_sim_data()
model.train_model(sim_data)

# 定義川普關稅戰情境(假設參數)
trump_scenario = {
    'tariff_increase': 25,      # 關稅大幅增加25個百分點
    'gdp_growth': -1.2,         # 全球GDP萎縮1.2%
    'trade_dependency': 45,     # 貿易依存度降至45%
    'debt_gdp_ratio': 120,      # 債務/GDP比率升至120%
    'vix': 45,                  # 恐慌指數飆升至45
    'yield_curve': -0.5         # 殖利率曲線倒掛0.5%
}

# 計算危機機率
probability = model.predict_depression_prob(trump_scenario)
print(f"全球經濟大蕭條發生機率：{probability:.1%}")

# 特徵重要性分析
importance = pd.Series(model.model.coef_[0], index=model.factors)
print("\n因子影響力分析(標準化係數)：")
print(importance.sort_values(ascending=False))

全球經濟大蕭條發生機率：100.0%

因子影響力分析(標準化係數)：
tariff_increase     4.493162
debt_gdp_ratio      1.637866
vix                 0.682702
yield_curve        -1.236791
trade_dependency   -2.585721
gdp_growth         -2.644087
dtype: float64
